In [1]:
# Install a set of compatible versions. Run in one cell.
!pip install -q "huggingface_hub==0.28.1" "transformers==4.41.0" "accelerate==0.20.3" ftfy safetensors
!pip install -q datasets torchvision pillow tqdm requests pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.1/464.1 kB 12.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 75.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 60.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.

In [2]:
import huggingface_hub, transformers, accelerate, torch
print("huggingface_hub", huggingface_hub.__version__)
print("transformers", transformers.__version__)
print("accelerate", accelerate.__version__)
print("torch", torch.__version__)


2025-10-18 03:44:57.703954: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760759097.952863      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760759098.029460      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `ty

huggingface_hub 0.28.1
transformers 4.41.0
accelerate 0.20.3
torch 2.6.0+cu124


In [4]:
from pathlib import Path
import json
import numpy as np
from tqdm.auto import tqdm
import os
import requests
from io import BytesIO
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import torch
import pandas as pd

print("torch:", torch.__version__)

torch: 2.6.0+cu124


In [5]:
# If running on Kaggle, dataset may be under /kaggle/input
POSSIBLE_LOCAL = Path("/kaggle/input")
if POSSIBLE_LOCAL.exists() and any(POSSIBLE_LOCAL.iterdir()):
    # pick CSV under /kaggle/input if present
    csvs = list(POSSIBLE_LOCAL.rglob("*.csv"))
    if csvs:
        DATA_FILE = csvs[0]
    else:
        # fallback to user-provided common path
        DATA_FILE = Path("/kaggle/working/ikarus.csv")
else:
    # local notebook path where you uploaded the file
    DATA_FILE = Path("/kaggle/working/intern_data_ikarus.csv")
    # if not present, check /mnt/data
    if not DATA_FILE.exists():
        alt = Path("/mnt/data/intern_data_ikarus.csv")
        if alt.exists():
            DATA_FILE = alt

if not DATA_FILE.exists():
    raise FileNotFoundError(f"Dataset file not found. Expected at {DATA_FILE} or /kaggle/input/* . Please upload the CSV to the notebook.")
print("Using dataset:", DATA_FILE)

Using dataset: /kaggle/input/ikarus/intern_data_ikarus.csv


In [6]:
df = pd.read_csv(DATA_FILE, low_memory=False)
print("Loaded dataframe shape:", df.shape)
print("Columns:", df.columns.tolist())

# Required columns the assignment mentions
required = ["title","brand","description","price","categories","images","manufacturer",
            "package dimensions","country_of_origin","material","color","uniq_id"]
missing = [c for c in required if c not in df.columns]
if missing:
    print("Warning - these required columns are missing from CSV:", missing)
else:
    print("All required columns are present.")

# Show a few rows for manual check
display(df.head(3))


Loaded dataframe shape: (312, 12)
Columns: ['title', 'brand', 'description', 'price', 'categories', 'images', 'manufacturer', 'package_dimensions', 'country_of_origin', 'material', 'color', 'uniq_id']
Warning - these required columns are missing from CSV: ['package dimensions']


,title,brand,description,price,categories,images,manufacturer,package_dimensions,country_of_origin,material,color,uniq_id
0,"GOYMFK 1pc Free Standing Shoe Rack, Multi-laye...",GOYMFK,"multiple shoes, coats, hats, and other items E...",$24.99,"['Home & Kitchen', 'Storage & Organization', '...",['https://m.media-amazon.com/images/I/416WaLx1...,GOYMFK,"2.36""D x 7.87""W x 21.6""H",China,Metal,White,02593e81-5c09-5069-8516-b0b29f439ded
1,"subrtex Leather ding Room, Dining Chairs Set o...",subrtex,subrtex Dining chairs Set of 2,NaN,"['Home & Kitchen', 'Furniture', 'Dining Room F...",['https://m.media-amazon.com/images/I/31SejUEW...,Subrtex Houseware INC,"18.5""D x 16""W x 35""H",NaN,Sponge,Black,5938d217-b8c5-5d3e-b1cf-e28e340f292e
2,Plant Repotting Mat MUYETOL Waterproof Transpl...,MUYETOL,NaN,$5.98,"['Patio, Lawn & Garden', 'Outdoor Décor', 'Doo...",['https://m.media-amazon.com/images/I/41RgefVq...,MUYETOL,"26.8""L x 26.8""W",NaN,Polyethylene,Green,b2ede786-3f51-5a45-9a5b-bcf856958cd8


In [7]:
# Normalize column names if data uses slight variations
# choose exact column names from df where possible
cols_lower = {c.lower(): c for c in df.columns}
def pick(colname):
    return cols_lower.get(colname, None)

title_col = pick("title") or pick("name")
desc_col = pick("description")
images_col = pick("images") or pick("image") or pick("image_url") or pick("imagepath")
uniq_col = pick("uniq_id") or pick("id") or pick("product_id")

if uniq_col is None:
    # fallback to row index as id
    df["uniq_id"] = df.index.astype(str)
    uniq_col = "uniq_id"

print("Mapped columns -> uniq:", uniq_col, "title:", title_col, "desc:", desc_col, "images:", images_col)

# build products list
products = []
for _, row in df.iterrows():
    pid = str(row.get(uniq_col)) if pd.notna(row.get(uniq_col)) else None
    title = str(row.get(title_col)) if title_col and pd.notna(row.get(title_col)) else ""
    desc = str(row.get(desc_col)) if desc_col and pd.notna(row.get(desc_col)) else ""
    images_val = None
    if images_col and pd.notna(row.get(images_col)):
        images_val = str(row.get(images_col))
    # preserve all metadata but do not drop important columns
    meta = {k: (None if pd.isna(v) else v) for k, v in row.to_dict().items()}
    products.append({"id": pid, "title": title, "description": desc, "images_field": images_val, "meta": meta})
products = [p for p in products if p["id"] is not None]
print("Prepared products count:", len(products))


Mapped columns -> uniq: uniq_id title: title desc: description images: images
Prepared products count: 312


In [8]:
WORK_DIR = Path("/kaggle/working")
IMAGES_DIR = WORK_DIR / "images"
IMAGES_DIR.mkdir(parents=True, exist_ok=True)

def download_or_resolve_image(image_val, pid):
    if image_val is None:
        return None
    image_val = str(image_val).strip()
    if image_val.lower().startswith("http"):
        try:
            resp = requests.get(image_val, timeout=15)
            if resp.status_code == 200:
                img = Image.open(BytesIO(resp.content)).convert("RGB")
                outp = IMAGES_DIR / f"{pid}.jpg"
                img.save(outp)
                return str(outp)
            else:
                return None
        except Exception:
            return None
    else:
        # treat as local path inside mounted dataset (try common locations)
        p = Path(image_val)
        if not p.exists():
            # try relative search inside /kaggle/input
            possible = list(Path("/kaggle/input").rglob(Path(image_val).name)) if Path("/kaggle/input").exists() else []
            if possible:
                p = possible[0]
        if p.exists():
            try:
                img = Image.open(p).convert("RGB")
                outp = IMAGES_DIR / f"{pid}.jpg"
                img.save(outp)
                return str(outp)
            except Exception:
                return None
        else:
            return None

for p in tqdm(products, desc="resolving images"):
    val = p.get("images_field")
    local = download_or_resolve_image(val, p["id"])
    p["local_image"] = local

num_img = sum(1 for p in products if p["local_image"])
print(f"Images available for {num_img}/{len(products)} products")


resolving images:   0%|          | 0/312 [00:00<?, ?it/s]

Images available for 0/312 products


In [9]:
from collections import Counter
vals = [p.get("images_field") for p in products]
# show 30 samples (first non-null and last non-null)
non_null = [v for v in vals if v is not None and str(v).strip() != ""]
print("Total rows:", len(products))
print("Non-empty images_field count:", len(non_null))
print("Sample of first 30 non-empty values:")
for i, v in enumerate(non_null[:30]):
    print(i+1, "->", repr(v)[:300])
# show common prefixes or separators
seps = Counter()
for v in non_null[:200]:
    s = str(v)
    if "|" in s:
        seps["pipe |"] += 1
    if ";" in s:
        seps["semicolon ;"] += 1
    if "," in s and "http" in s:
        seps["comma , (with http)"] += 1
    if s.strip().startswith("[") and "http" in s:
        seps["looks like list"] += 1
    if "http" in s:
        seps["contains http"] += 1
print("Heuristic separators / features (counts):", dict(seps))


Total rows: 312
Non-empty images_field count: 312
Sample of first 30 non-empty values:
1 -> "['https://m.media-amazon.com/images/I/416WaLx10jL._SS522_.jpg ', ' https://m.media-amazon.com/images/I/41kuxipTsuL._SS522_.jpg ', ' https://m.media-amazon.com/images/I/51T9x4yZd3L._SS522_.jpg ', ' https://m.media-amazon.com/images/I/61w6ifIrCpL._SS522_.jpg ', ' https://m.media-amazon.com/images/I/5
2 -> "['https://m.media-amazon.com/images/I/31SejUEWY7L._SS522_.jpg ', ' https://m.media-amazon.com/images/I/41mr+A9JmbL._SS522_.jpg ', ' https://m.media-amazon.com/images/I/41JjrWgA0XL._SS522_.jpg ', ' https://m.media-amazon.com/images/I/41ie5FbnfkL._SS522_.jpg ', ' https://m.media-amazon.com/images/I/4
3 -> "['https://m.media-amazon.com/images/I/41RgefVq70L._SS522_.jpg ', ' https://m.media-amazon.com/images/I/414SPEuzxlL._SS522_.jpg ', ' https://m.media-amazon.com/images/I/51gknsPKCHL._SS522_.jpg ', ' https://m.media-amazon.com/images/I/51mcO9+8GLL._SS522_.jpg ', ' https://m.media-amazon.com/images

In [10]:
import re, json
from pathlib import Path

# extraction helper tuned for the dataset format
URL_REGEX = re.compile(r"https?://[^\s'\",]+")  # stop at space, quote or comma

def extract_first_candidate(s):
    if s is None:
        return None
    s = str(s).strip()
    if not s:
        return None
    # Try regex first (will find the first http... in the string)
    m = URL_REGEX.search(s)
    if m:
        return m.group(0)
    # handle single-quoted python-list string: "['url1','url2']"
    # remove surrounding quotes if present
    if (s.startswith("'") and s.endswith("'")) or (s.startswith('"') and s.endswith('"')):
        s2 = s[1:-1].strip()
    else:
        s2 = s
    # replace single quotes with double to try JSON loads
    try:
        if s2.startswith("[") and s2.endswith("]"):
            # convert single quotes -> double quotes safely for typical cases
            json_like = s2.replace("'", '"')
            parsed = json.loads(json_like)
            if isinstance(parsed, (list, tuple)) and parsed:
                # return first non-empty string trimmed
                for item in parsed:
                    if item and str(item).strip():
                        return str(item).strip()
    except Exception:
        pass
    # fallback: split on common separators
    for sep in ["|", ";", ","]:
        if sep in s2:
            parts = [x.strip().strip("'").strip('"') for x in s2.split(sep) if x.strip()]
            if parts:
                return parts[0]
    return s2

# show first 20 parsed candidates
samples = [p.get("images_field") for p in products][:50]
for i, s in enumerate(samples[:20]):
    print(i+1, "raw->", repr(s)[:200])
    print("   parsed->", extract_first_candidate(s))


1 raw-> "['https://m.media-amazon.com/images/I/416WaLx10jL._SS522_.jpg ', ' https://m.media-amazon.com/images/I/41kuxipTsuL._SS522_.jpg ', ' https://m.media-amazon.com/images/I/51T9x4yZd3L._SS522_.jpg ', ' ht
   parsed-> https://m.media-amazon.com/images/I/416WaLx10jL._SS522_.jpg
2 raw-> "['https://m.media-amazon.com/images/I/31SejUEWY7L._SS522_.jpg ', ' https://m.media-amazon.com/images/I/41mr+A9JmbL._SS522_.jpg ', ' https://m.media-amazon.com/images/I/41JjrWgA0XL._SS522_.jpg ', ' ht
   parsed-> https://m.media-amazon.com/images/I/31SejUEWY7L._SS522_.jpg
3 raw-> "['https://m.media-amazon.com/images/I/41RgefVq70L._SS522_.jpg ', ' https://m.media-amazon.com/images/I/414SPEuzxlL._SS522_.jpg ', ' https://m.media-amazon.com/images/I/51gknsPKCHL._SS522_.jpg ', ' ht
   parsed-> https://m.media-amazon.com/images/I/41RgefVq70L._SS522_.jpg
4 raw-> "['https://m.media-amazon.com/images/I/61vz1IglerL._SS522_.jpg ', ' https://m.media-amazon.com/images/I/41KiuOZ9IlL._SS522_.jpg ', ' https://m.media-

In [11]:
import requests
from io import BytesIO
from PIL import Image
from tqdm.auto import tqdm
import re, json
from pathlib import Path

WORK_DIR = Path("/kaggle/working")
IMAGES_DIR = WORK_DIR / "images"
IMAGES_DIR.mkdir(parents=True, exist_ok=True)

URL_REGEX = re.compile(r"https?://[^\s'\",]+")

def extract_first_url_from_field(s):
    """Return first URL or local token from images_field."""
    if s is None:
        return None
    s = str(s).strip()
    if s == "" or s.lower() in ("none","nan","[]"):
        return None
    # quick regex fetch first http(s) URL
    m = URL_REGEX.search(s)
    if m:
        return m.group(0).strip()
    # remove outer quotes
    if (s.startswith('"') and s.endswith('"')) or (s.startswith("'") and s.endswith("'")):
        inner = s[1:-1].strip()
    else:
        inner = s
    # try JSON-like: convert single quotes to double quotes if looks like list
    if inner.startswith("[") and inner.endswith("]"):
        try:
            candidate_list = json.loads(inner.replace("'", '"'))
            if isinstance(candidate_list, (list,tuple)) and candidate_list:
                # find first http-like element
                for it in candidate_list:
                    if it and isinstance(it, str) and "http" in it:
                        return it.strip()
                # otherwise return first element as fallback
                return str(candidate_list[0]).strip()
        except Exception:
            pass
    # split on common separators
    for sep in ["|",";"," ,",","]:
        if sep in inner:
            parts = [x.strip().strip("'").strip('"') for x in inner.split(sep) if x.strip()]
            for p in parts:
                if p and "http" in p:
                    return p
            if parts:
                return parts[0]
    return inner

def download_image_from_url(url, out_path, timeout=20):
    """Download and save an image; return True on success."""
    headers = {"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36"}
    try:
        resp = requests.get(url, timeout=timeout, headers=headers)
        if resp.status_code != 200:
            return False
        img = Image.open(BytesIO(resp.content)).convert("RGB")
        img.save(out_path)
        return True
    except Exception:
        return False

# Run resolver + downloader for all products; download first URL only
downloaded = 0
failed = 0
for p in tqdm(products, desc="resolving & downloading images"):
    raw = p.get("images_field")
    first = extract_first_url_from_field(raw)
    if not first:
        p["local_image"] = None
        continue
    # if candidate looks like URL -> download
    if str(first).lower().startswith("http"):
        outp = IMAGES_DIR / f"{p['id']}.jpg"
        ok = download_image_from_url(first, outp)
        if ok:
            p["local_image"] = str(outp)
            downloaded += 1
        else:
            p["local_image"] = None
            failed += 1
    else:
        # treat as local filename inside /kaggle/input
        cand = Path(first)
        found = None
        if cand.exists():
            found = cand
        else:
            # search under /kaggle/input
            if Path("/kaggle/input").exists():
                possible = list(Path("/kaggle/input").rglob(cand.name))
                if possible:
                    found = possible[0]
        if found:
            try:
                img = Image.open(found).convert("RGB")
                outp = IMAGES_DIR / f"{p['id']}.jpg"
                img.save(outp)
                p["local_image"] = str(outp)
                downloaded += 1
            except Exception:
                p["local_image"] = None
                failed += 1
        else:
            p["local_image"] = None
            failed += 1

print("Downloaded:", downloaded, "Failed attempts:", failed)


resolving & downloading images:   0%|          | 0/312 [00:00<?, ?it/s]

Downloaded: 312 Failed attempts: 0


In [12]:

num_img = sum(1 for p in products if p.get("local_image"))
print(f"Images available for {num_img}/{len(products)} products")

# show 10 sample local_image paths
cnt = 0
for p in products:
    if p.get("local_image"):
        print(p["id"], "->", p["local_image"])
        cnt += 1
        if cnt >= 10:
            break


Images available for 312/312 products
02593e81-5c09-5069-8516-b0b29f439ded -> /kaggle/working/images/02593e81-5c09-5069-8516-b0b29f439ded.jpg
5938d217-b8c5-5d3e-b1cf-e28e340f292e -> /kaggle/working/images/5938d217-b8c5-5d3e-b1cf-e28e340f292e.jpg
b2ede786-3f51-5a45-9a5b-bcf856958cd8 -> /kaggle/working/images/b2ede786-3f51-5a45-9a5b-bcf856958cd8.jpg
8fd9377b-cfa6-5f10-835c-6b8eca2816b5 -> /kaggle/working/images/8fd9377b-cfa6-5f10-835c-6b8eca2816b5.jpg
bdc9aa30-9439-50dc-8e89-213ea211d66a -> /kaggle/working/images/bdc9aa30-9439-50dc-8e89-213ea211d66a.jpg
20da3703-26f1-53e5-aa0b-a8104527d1bb -> /kaggle/working/images/20da3703-26f1-53e5-aa0b-a8104527d1bb.jpg
aba4138e-6401-52ca-a099-02e30b638db4 -> /kaggle/working/images/aba4138e-6401-52ca-a099-02e30b638db4.jpg
02593e81-5c09-5069-8516-b0b29f439ded -> /kaggle/working/images/02593e81-5c09-5069-8516-b0b29f439ded.jpg
5938d217-b8c5-5d3e-b1cf-e28e340f292e -> /kaggle/working/images/5938d217-b8c5-5d3e-b1cf-e28e340f292e.jpg
b2ede786-3f51-5a45-9a5b-bc

In [13]:
# Load CLIP model + helper functions
import torch
from transformers import CLIPProcessor, CLIPModel
from pathlib import Path
import numpy as np
from tqdm.auto import tqdm
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

MODEL_NAME = "openai/clip-vit-base-patch32"
print("Loading CLIP model:", MODEL_NAME)
model = CLIPModel.from_pretrained(MODEL_NAME).to(device)
processor = CLIPProcessor.from_pretrained(MODEL_NAME)
model.eval()

# helpers for batching & normalization
def l2_normalize_rows(a: np.ndarray):
    if a.size == 0:
        return a
    norms = np.linalg.norm(a, axis=1, keepdims=True)
    norms[norms == 0] = 1.0
    return a / norms

def compute_text_embeddings(texts, batch_size=64):
    all_emb = []
    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size), desc="text batches"):
            batch = texts[i:i+batch_size]
            inputs = processor(text=batch, return_tensors="pt", padding=True).to(device)
            emb = model.get_text_features(**inputs)   # (B, D)
            all_emb.append(emb.cpu().numpy())
    if not all_emb:
        return np.zeros((0, model.config.projection_dim), dtype=np.float32)
    return np.vstack(all_emb).astype(np.float32)

def compute_image_embeddings(paths, batch_size=32):
    all_emb = []
    with torch.no_grad():
        for i in tqdm(range(0, len(paths), batch_size), desc="image batches"):
            batch_paths = paths[i:i+batch_size]
            images = []
            for p in batch_paths:
                if p is None:
                    images.append(Image.new("RGB",(224,224),color=(128,128,128)))
                else:
                    try:
                        img = Image.open(p).convert("RGB")
                    except Exception:
                        img = Image.new("RGB",(224,224),color=(128,128,128))
                    images.append(img)
            inputs = processor(images=images, return_tensors="pt", padding=True).to(device)
            emb = model.get_image_features(**inputs)  # (B, D)
            all_emb.append(emb.cpu().numpy())
    if not all_emb:
        return np.zeros((0, model.config.projection_dim), dtype=np.float32)
    return np.vstack(all_emb).astype(np.float32)


Device: cpu
Loading CLIP model: openai/clip-vit-base-patch32


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [15]:
# Chunk-aware text embedding + recompute embeddings / index
import math
import numpy as np
from tqdm.auto import tqdm
import torch

# Access tokenizer and model
tokenizer = processor.tokenizer
# typical CLIP max length; fallback to 77
max_len = getattr(tokenizer, "model_max_length", None) or 77
print("Tokenizer max length:", max_len)

# Create chunks per text based on token length
def make_text_chunks(texts, max_tokens=max_len):
    """
    For each text in texts list produce one or more chunks (strings), 
    and build a mapping from chunk index to original text index.
    Returns: chunks_list, mapping (list of original indices for each chunk)
    """
    chunks = []
    mapping = []  # mapping[i_chunk] = index_of_original_text
    for idx, txt in enumerate(texts):
        if not txt:
            # empty text -> one empty chunk
            chunks.append("")
            mapping.append(idx)
            continue
        # quick token-length check
        enc = tokenizer(txt, truncation=False, add_special_tokens=True)
        length = len(enc["input_ids"])
        if length <= max_tokens:
            chunks.append(txt)
            mapping.append(idx)
            continue
        # else, split into word pieces approximately and produce chunks
        words = txt.split()
        cur_chunk_words = []
        # iterate words, accumulate until tokenized length would exceed max_tokens-2
        for w in words:
            cur_chunk_words.append(w)
            # check token count of current chunk
            cur_txt = " ".join(cur_chunk_words)
            cur_len = len(tokenizer(cur_txt, truncation=False, add_special_tokens=True)["input_ids"])
            if cur_len >= max_tokens:
                # back off last word to keep under limit
                # if single word too long (rare), we still take it
                if len(cur_chunk_words) == 1:
                    chunks.append(cur_txt)
                    mapping.append(idx)
                    cur_chunk_words = []
                else:
                    # remove last word, finalize chunk
                    last = cur_chunk_words.pop()
                    chunk_txt = " ".join(cur_chunk_words)
                    chunks.append(chunk_txt)
                    mapping.append(idx)
                    # start new chunk with last word
                    cur_chunk_words = [last]
        # any remaining words -> final chunk
        if cur_chunk_words:
            chunks.append(" ".join(cur_chunk_words))
            mapping.append(idx)
    return chunks, mapping

# Build texts list again (just in case)
texts = [(p.get("title","") + ". " + (p.get("description") or "")) for p in products]

print("Preparing chunks for", len(texts), "texts...")
chunks, mapping = make_text_chunks(texts, max_tokens=max_len)
print("Total chunks produced:", len(chunks))

# Now compute embeddings for all chunks in batches
def compute_text_embeddings_for_chunks(chunks_list, batch_size=64):
    all_emb = []
    with torch.no_grad():
        for i in tqdm(range(0, len(chunks_list), batch_size), desc="chunk batches"):
            batch = chunks_list[i:i+batch_size]
            inputs = processor(text=batch, return_tensors="pt", padding=True, truncation=True, max_length=max_len).to(device)
            emb = model.get_text_features(**inputs)  # (B, D)
            all_emb.append(emb.cpu().numpy())
    if not all_emb:
        return np.zeros((0, model.config.projection_dim), dtype=np.float32)
    return np.vstack(all_emb).astype(np.float32)

chunk_emb = compute_text_embeddings_for_chunks(chunks, batch_size=64)
print("Chunk embeddings shape:", chunk_emb.shape)

# Average chunk embeddings per original text index
dim = chunk_emb.shape[1]
n_texts = len(texts)
text_emb_new = np.zeros((n_texts, dim), dtype=np.float32)
counts = np.zeros((n_texts,), dtype=np.int32)
for i_chunk, orig_idx in enumerate(mapping):
    text_emb_new[orig_idx] += chunk_emb[i_chunk]
    counts[orig_idx] += 1
# avoid divide-by-zero
for i in range(n_texts):
    if counts[i] > 0:
        text_emb_new[i] = text_emb_new[i] / counts[i]
    else:
        # fallback: zero vector (shouldn't happen)
        text_emb_new[i] = np.zeros((dim,), dtype=np.float32)

# Normalize
def l2_normalize_rows_np(a):
    norms = np.linalg.norm(a, axis=1, keepdims=True)
    norms[norms==0] = 1.0
    return a / norms

text_emb_new = l2_normalize_rows_np(text_emb_new)
print("Final text embeddings shape (per product):", text_emb_new.shape)

# Recompute image embeddings if not yet computed or if you want to reuse existing img_emb
# If img_emb variable exists and has correct shape, we can reuse it. Otherwise compute:
try:
    img_emb  # check existence
    recompute_images = False
except NameError:
    recompute_images = True

if recompute_images:
    print("Computing image embeddings (images were not present in memory)...")
    img_emb = compute_image_embeddings(image_paths, batch_size=32)
    img_emb = l2_normalize_rows_np(img_emb)
    print("Image embeddings shape:", img_emb.shape)
else:
    print("Reusing existing img_emb with shape:", img_emb.shape)

# Combine embeddings exactly like before
w_text = 0.6
w_img = 0.4

combined = []
ids = []
metadata = {}
missing_image_count = 0

for i, p in enumerate(products):
    te = text_emb_new[i] if text_emb_new.shape[0] > i else np.zeros((dim,), dtype=np.float32)
    ie = img_emb[i] if img_emb.shape[0] > i else np.zeros((dim,), dtype=np.float32)
    if p.get("local_image") is None or (ie is None or ie.size == 0):
        v = te
        missing_image_count += 1
    else:
        v = (w_text * te + w_img * ie)
    v = v / (np.linalg.norm(v) + 1e-12)
    combined.append(v.astype(np.float32))
    pid = str(p.get("id") or p.get("uniq_id") or i)
    ids.append(pid)
    meta = p.get("meta", {}).copy()
    meta.pop("local_image", None)
    metadata[pid] = meta

matrix = np.vstack(combined)
print("Combined matrix shape:", matrix.shape)
print("Products without image (used text only):", missing_image_count)

# Save again (overwrite previous)
import json, os
OUT_DIR = Path("/kaggle/working/index")
OUT_DIR.mkdir(parents=True, exist_ok=True)
index_path = OUT_DIR / "index.npz"
meta_path = OUT_DIR / "index.npz.meta.json"

np.savez_compressed(str(index_path), ids=np.array(ids, dtype=object), matrix=matrix)
with open(meta_path, "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

print("Saved index to:", index_path)
print("Saved metadata to:", meta_path)


Tokenizer max length: 77
Preparing chunks for 312 texts...
Total chunks produced: 572


chunk batches:   0%|          | 0/9 [00:00<?, ?it/s]

Chunk embeddings shape: (572, 512)
Final text embeddings shape (per product): (312, 512)
Computing image embeddings (images were not present in memory)...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:123                                                                            │
│                                                                                                  │
│   120                                                                                            │
│   121 if recompute_images:                                                                       │
│   122 │   print("Computing image embeddings (images were not present in memory)...")             │
│ ❱ 123 │   img_emb = compute_image_embeddings(image_paths, batch_size=32)                         │
│   124 │   img_emb = l2_normalize_rows_np(img_emb)                                                │
│   125 │   print("Image embeddings shape:", img_emb.shape)                                        │
│   126 else:                                                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'image_paths' is not defined

In [16]:
# Fix: define image_paths, compute image embeddings, recombine & save index
from pathlib import Path
import numpy as np, json
from tqdm.auto import tqdm
from PIL import Image
import torch

# 1) ensure image_paths exists
image_paths = [p.get("local_image") for p in products]
print("Example image paths (first 8):", image_paths[:8])
print("Count image paths:", len(image_paths))

# 2) ensure compute_image_embeddings exists (define fallback safe version if not)
try:
    compute_image_embeddings
except NameError:
    # define a safe compute_image_embeddings (same as earlier, but included here to be robust)
    def compute_image_embeddings(paths, batch_size=16):
        all_emb = []
        with torch.no_grad():
            for i in tqdm(range(0, len(paths), batch_size), desc="image batches"):
                batch_paths = paths[i:i+batch_size]
                images = []
                for p in batch_paths:
                    if p is None:
                        images.append(Image.new("RGB",(224,224),color=(128,128,128)))
                    else:
                        try:
                            img = Image.open(p).convert("RGB")
                        except Exception:
                            img = Image.new("RGB",(224,224),color=(128,128,128))
                        images.append(img)
                inputs = processor(images=images, return_tensors="pt", padding=True).to(device)
                emb = model.get_image_features(**inputs)  # (B, D)
                all_emb.append(emb.cpu().numpy())
        if not all_emb:
            return np.zeros((0, model.config.projection_dim), dtype=np.float32)
        return np.vstack(all_emb).astype(np.float32)

# 3) compute image embeddings (use smaller batch if memory issues)
batch_size_images = 16  # lower this to 8 if you get OOM
print("Computing image embeddings with batch_size =", batch_size_images)
img_emb = compute_image_embeddings(image_paths, batch_size=batch_size_images)

# 4) normalize image embeddings
def l2_normalize_rows_np(a):
    if a is None or a.size == 0:
        return a
    norms = np.linalg.norm(a, axis=1, keepdims=True)
    norms[norms==0] = 1.0
    return a / norms

img_emb = l2_normalize_rows_np(img_emb)
print("Image embeddings shape:", None if img_emb is None else img_emb.shape)

# 5) Recombine with text_emb_new (must exist from previous chunking)
if 'text_emb_new' not in globals():
    raise RuntimeError("text_emb_new not found — rerun the chunked text embedding cell first.")
text_emb = text_emb_new  # alias

# weights
w_text = 0.6
w_img = 0.4

combined = []
ids = []
metadata = {}
missing_image_count = 0
dim = text_emb.shape[1]

# If img_emb has fewer rows than products, align by index (it should have same length)
if img_emb.shape[0] != len(products):
    print("Warning: img_emb rows != number of products. Filling missing entries with zeros.")
    # build img_emb_full with zeros then fill available rows
    img_full = np.zeros((len(products), dim), dtype=np.float32)
    n_fill = min(img_emb.shape[0], len(products))
    img_full[:n_fill] = img_emb[:n_fill]
    img_emb = img_full

for i, p in enumerate(products):
    te = text_emb[i] if text_emb.shape[0] > i else np.zeros((dim,), dtype=np.float32)
    ie = img_emb[i] if img_emb.shape[0] > i else np.zeros((dim,), dtype=np.float32)
    if p.get("local_image") is None or (ie is None or ie.size == 0):
        v = te
        missing_image_count += 1
    else:
        v = (w_text * te + w_img * ie)
    v = v / (np.linalg.norm(v) + 1e-12)
    combined.append(v.astype(np.float32))
    pid = str(p.get("id") or p.get("uniq_id") or i)
    ids.append(pid)
    meta = p.get("meta", {}).copy()
    meta.pop("local_image", None)
    metadata[pid] = meta

matrix = np.vstack(combined)
print("Combined matrix shape:", matrix.shape)
print("Products without image (used text only):", missing_image_count)

# 6) Save index & metadata
OUT_DIR = Path("/kaggle/working/index")
OUT_DIR.mkdir(parents=True, exist_ok=True)
index_path = OUT_DIR / "index.npz"
meta_path = OUT_DIR / "index.npz.meta.json"

np.savez_compressed(str(index_path), ids=np.array(ids, dtype=object), matrix=matrix)
with open(meta_path, "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

print("Saved index to:", index_path)
print("Saved metadata to:", meta_path)

# 7) quick verification: load back and sample nearest neighbors for a sample prompt
data = np.load(str(index_path), allow_pickle=True)
print("Saved file entries:", list(data.keys()))
mat_loaded = data["matrix"]
print("Loaded matrix shape:", mat_loaded.shape)


Example image paths (first 8): ['/kaggle/working/images/02593e81-5c09-5069-8516-b0b29f439ded.jpg', '/kaggle/working/images/5938d217-b8c5-5d3e-b1cf-e28e340f292e.jpg', '/kaggle/working/images/b2ede786-3f51-5a45-9a5b-bcf856958cd8.jpg', '/kaggle/working/images/8fd9377b-cfa6-5f10-835c-6b8eca2816b5.jpg', '/kaggle/working/images/bdc9aa30-9439-50dc-8e89-213ea211d66a.jpg', '/kaggle/working/images/20da3703-26f1-53e5-aa0b-a8104527d1bb.jpg', '/kaggle/working/images/aba4138e-6401-52ca-a099-02e30b638db4.jpg', '/kaggle/working/images/02593e81-5c09-5069-8516-b0b29f439ded.jpg']
Count image paths: 312
Computing image embeddings with batch_size = 16


image batches:   0%|          | 0/20 [00:00<?, ?it/s]

Image embeddings shape: (312, 512)
Combined matrix shape: (312, 512)
Products without image (used text only): 0
Saved index to: /kaggle/working/index/index.npz
Saved metadata to: /kaggle/working/index/index.npz.meta.json
Saved file entries: ['ids', 'matrix']
Loaded matrix shape: (312, 512)


In [17]:
# Evaluation utilities + load index & meta if not in memory
import numpy as np, json, math
from pathlib import Path
from collections import defaultdict, Counter

WORK_DIR = Path("/kaggle/working")
INDEX_DIR = WORK_DIR / "index"
INDEX_PATH = INDEX_DIR / "index.npz"
META_PATH = INDEX_DIR / "index.npz.meta.json"

# try to reuse in-memory objects, otherwise load from disk
try:
    ids  # check existence
    print("Using in-memory 'ids' and 'matrix' variables")
except NameError:
    print("Loading index from disk:", INDEX_PATH)
    data = np.load(str(INDEX_PATH), allow_pickle=True)
    ids = data["ids"].tolist()
    matrix = data["matrix"].astype(np.float32)

# load metadata (if not in memory)
try:
    metadata
except NameError:
    if META_PATH.exists():
        with open(META_PATH, "r", encoding="utf-8") as f:
            metadata = json.load(f)
        print("Loaded metadata entries:", len(metadata))
    else:
        metadata = {}
        print("No meta json found; metadata empty.")

# helper mapping id -> index
id_to_index = {str(pid): idx for idx, pid in enumerate(ids)}
n_items = len(ids)
print("n_items:", n_items)


Using in-memory 'ids' and 'matrix' variables
n_items: 312


In [18]:
import re

# Parse categories string into tokens
def parse_category_tokens(cat_val):
    """
    Turn various category formats into a set of normalized tokens.
    Handles strings like 'Electronics > Mice', "'Home|Kitchen'", "['A','B']" etc.
    """
    if cat_val is None:
        return set()
    s = str(cat_val).strip()
    if s == "" or s.lower() in ("nan","none","[]"):
        return set()
    # remove outer quotes if present
    if (s.startswith('"') and s.endswith('"')) or (s.startswith("'") and s.endswith("'")):
        s = s[1:-1].strip()
    # try to replace JSON-like single quotes and parse lists
    if s.startswith("[") and s.endswith("]"):
        try:
            import json
            parsed = json.loads(s.replace("'", '"'))
            if isinstance(parsed, (list, tuple)):
                tokens = []
                for it in parsed:
                    if it:
                        tokens.extend(re.split(r"[>|/;,]", str(it)))
                tokens = [t.strip().lower() for t in tokens if t and t.strip()]
                return set(tokens)
        except Exception:
            pass
    # split on separators common in scraped data
    parts = re.split(r"[>|/;,|\t\n]", s)
    parts = [p.strip().lower() for p in parts if p and p.strip()]
    return set(parts)

# build tokens per item using metadata "categories" or fallback to brand/manufacturer
category_tokens = {}
for pid in ids:
    meta = metadata.get(str(pid), {})
    # attempted keys
    catraw = None
    for k in ("categories","category","cat","product_categories"):
        if k in meta:
            catraw = meta[k]
            break
    tokens = parse_category_tokens(catraw)
    # fallback: brand or manufacturer token
    if not tokens:
        for k in ("brand","manufacturer"):
            if k in meta and meta[k]:
                tokens = {str(meta[k]).strip().lower()}
                break
    category_tokens[str(pid)] = tokens

# quick stats
counts = [len(t) for t in category_tokens.values()]
import numpy as np
print("category tokens per item: mean=%.2f median=%d (min=%d max=%d)" % (np.mean(counts), int(np.median(counts)), int(np.min(counts)), int(np.max(counts))))
# how many items with zero tokens
zero_tokens = sum(1 for t in category_tokens.values() if len(t)==0)
print("Items with zero tokens (no categories/brand/manufacturer):", zero_tokens)


category tokens per item: mean=4.44 median=4 (min=2 max=6)
Items with zero tokens (no categories/brand/manufacturer): 0


In [19]:
# For each item, ground truth = other items that share >=1 token in category_tokens
gt_sets = {}
for pid in ids:
    toks = category_tokens.get(str(pid), set())
    if not toks:
        gt_sets[pid] = set()  # no ground truth available
        continue
    related = set()
    # naive O(N) scan (OK for n~thousands; adapt to inverted index if larger)
    for other in ids:
        if other == pid:
            continue
        oth_toks = category_tokens.get(str(other), set())
        if toks & oth_toks:
            related.add(other)
    gt_sets[pid] = related

# Quick distribution of ground truth sizes
sizes = [len(s) for s in gt_sets.values()]
from collections import Counter
ctr = Counter(sizes)
print("GT size distribution (counts->num items):", {k:v for k,v in sorted(ctr.items())[:10]})
print("Examples: product id, gt size (first 10):")
for pid in list(ids)[:10]:
    print(pid, len(gt_sets[pid]))


GT size distribution (counts->num items): {1: 2, 5: 6, 12: 13, 16: 17, 20: 20, 24: 4, 35: 1, 191: 3, 214: 3, 235: 6}
Examples: product id, gt size (first 10):
02593e81-5c09-5069-8516-b0b29f439ded 238
5938d217-b8c5-5d3e-b1cf-e28e340f292e 242
b2ede786-3f51-5a45-9a5b-bcf856958cd8 20
8fd9377b-cfa6-5f10-835c-6b8eca2816b5 20
bdc9aa30-9439-50dc-8e89-213ea211d66a 241
20da3703-26f1-53e5-aa0b-a8104527d1bb 241
aba4138e-6401-52ca-a099-02e30b638db4 241
02593e81-5c09-5069-8516-b0b29f439ded 238
5938d217-b8c5-5d3e-b1cf-e28e340f292e 242
b2ede786-3f51-5a45-9a5b-bcf856958cd8 20


In [20]:
# retrieval helpers: compute Recall@K and MRR for a matrix and ground truth sets
import numpy as np

def recall_at_k_for_all(query_matrix, target_matrix, topk_list=[1,5,10]):
    """
    query_matrix: (N, D) numpy (one query per product) - rows normalized
    target_matrix: (N, D) numpy (same index order as ids) - rows normalized
    returns dict with recall@k and mrr
    """
    N, D = target_matrix.shape
    assert query_matrix.shape[0] == N
    # compute similarity via dot product (rows normalized)
    # sims matrix could be large O(N^2) but N=312 here is fine. For larger N, do batch.
    sims = query_matrix @ target_matrix.T  # (N, N)
    # set self-sim to -inf so we don't return itself as match
    for i in range(N):
        sims[i, i] = -999.0
    results = {k: 0 for k in topk_list}
    mrr_sum = 0.0
    valid_queries = 0
    for i, pid in enumerate(ids):
        gt = gt_sets.get(pid, set())
        if not gt:
            # skip queries without ground truth
            continue
        valid_queries += 1
        row = sims[i]
        # get sorted indices desc
        idxs = np.argsort(-row)
        # compute ranks of the first hit
        hit_rank = None
        for rank, idx in enumerate(idxs, start=1):
            if ids[int(idx)] in gt:
                hit_rank = rank
                break
        if hit_rank is None:
            # no hit in whole catalog
            pass
        else:
            # update recalls
            for k in topk_list:
                if hit_rank <= k:
                    results[k] += 1
            mrr_sum += 1.0 / float(hit_rank)
    # finalize metrics
    if valid_queries == 0:
        return {"valid_queries":0}
    res = {}
    for k in topk_list:
        res[f"recall@{k}"] = results[k] / valid_queries
    res["MRR"] = mrr_sum / valid_queries
    res["valid_queries"] = valid_queries
    return res


In [21]:
# Prepare matrices: try to reuse in-memory text_emb_new, img_emb; otherwise recompute or fallback
# 'matrix' variable should be the combined matrix we saved earlier (rows normalized)
try:
    combined_matrix = matrix  # from disk or earlier
except NameError:
    combined_matrix = np.load(str(INDEX_PATH), allow_pickle=True)["matrix"].astype(np.float32)

# ensure normalized
def normalize_rows(a):
    norms = np.linalg.norm(a, axis=1, keepdims=True)
    norms[norms==0] = 1.0
    return a / norms

combined_matrix = normalize_rows(combined_matrix)

# text-only
try:
    text_matrix = text_emb_new  # computed by the chunking step earlier
    print("Using in-memory text_emb_new with shape", text_matrix.shape)
except NameError:
    # attempt to compute quick text-only by encoding titles only (faster but less accurate)
    print("text_emb_new not found in memory. Recomputing text-only using titles+desc chunking...")
    # shortened re-run of chunking -> compute embeddings (could take time)
    raise RuntimeError("text_emb_new missing — rerun chunked embedding cell or let me compute text-only now.")

text_matrix = normalize_rows(text_matrix)

# image-only
try:
    img_matrix = img_emb  # from earlier compute_image_embeddings
    print("Using in-memory img_emb with shape", img_matrix.shape)
except NameError:
    # If image-only not available, fallback to zeros
    print("img_emb missing; building zero-image matrix")
    img_matrix = np.zeros_like(text_matrix)

img_matrix = normalize_rows(img_matrix)

# compute metrics (Recall@1,5,10 and MRR)
topk = [1,5,10]
print("Evaluating combined...")
combined_res = recall_at_k_for_all(combined_matrix, combined_matrix, topk_list=topk)
print("Combined:", combined_res)

print("Evaluating text-only...")
text_res = recall_at_k_for_all(text_matrix, text_matrix, topk_list=topk)
print("Text-only:", text_res)

print("Evaluating image-only...")
img_res = recall_at_k_for_all(img_matrix, img_matrix, topk_list=topk)
print("Image-only:", img_res)


Using in-memory text_emb_new with shape (312, 512)
Using in-memory img_emb with shape (312, 512)
Evaluating combined...
Combined: {'recall@1': 0.8621794871794872, 'recall@5': 0.9807692307692307, 'recall@10': 0.9871794871794872, 'MRR': 0.9166361326608706, 'valid_queries': 312}
Evaluating text-only...
Text-only: {'recall@1': 0.8525641025641025, 'recall@5': 0.9711538461538461, 'recall@10': 0.9839743589743589, 'MRR': 0.9088039809857403, 'valid_queries': 312}
Evaluating image-only...
Image-only: {'recall@1': 0.8237179487179487, 'recall@5': 0.9583333333333334, 'recall@10': 0.9743589743589743, 'MRR': 0.8854238907027263, 'valid_queries': 312}


In [22]:
# Compute per-category (top token) average Recall@5 to see which categories perform well
from collections import defaultdict
cat_to_pids = defaultdict(list)
for pid, toks in category_tokens.items():
    if not toks:
        continue
    # pick primary token as first sorted token (heuristic)
    primary = sorted(toks)[0]
    cat_to_pids[primary].append(pid)

def recall_at_k_for_pidlist(pid_list, matrix, k=5):
    # compute recall for this set of queries only
    if not pid_list:
        return None
    sub_q_idx = [id_to_index[pid] for pid in pid_list if pid in id_to_index]
    if not sub_q_idx:
        return None
    # build sub query matrix and reuse full matrix as target
    qmat = matrix[sub_q_idx]
    res = recall_at_k_for_all(qmat, matrix, topk_list=[k])
    return res

# evaluate top categories with >= 5 items
cat_scores = []
for cat, pids in cat_to_pids.items():
    if len(pids) < 5:
        continue
    res = recall_at_k_for_pidlist(pids, combined_matrix, k=5)
    if res and res.get("valid_queries",0)>0:
        cat_scores.append((cat, len(pids), res["recall@5"]))
cat_scores_sorted = sorted(cat_scores, key=lambda x: -x[2])
print("Top categories by recall@5 (combined):")
for cat, cnt, score in cat_scores_sorted[:15]:
    print(f"{cat}: items={cnt}, recall@5={score:.3f}")


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:28                                                                             │
│                                                                                                  │
│   25 for cat, pids in cat_to_pids.items():                                                       │
│   26 │   if len(pids) < 5:                                                                       │
│   27 │   │   continue                                                                            │
│ ❱ 28 │   res = recall_at_k_for_pidlist(pids, combined_matrix, k=5)                               │
│   29 │   if res and res.get("valid_queries",0)>0:                                                │
│   30 │   │   cat_scores.append((cat, len(pids), res["recall@5"]))                                │
│   31 cat_scores_sorted = sorted(cat_scores, key=lambda x: -x[2])                                 │
│                                                                                                  │
│ in recall_at_k_for_pidlist:20                                                                    │
│                                                                                                  │
│   17 │   │   return None                                                                         │
│   18 │   # build sub query matrix and reuse full matrix as target                                │
│   19 │   qmat = matrix[sub_q_idx]                                                                │
│ ❱ 20 │   res = recall_at_k_for_all(qmat, matrix, topk_list=[k])                                  │
│   21 │   return res                                                                              │
│   22                                                                                             │
│   23 # evaluate top categories with >= 5 items                                                   │
│                                                                                                  │
│ in recall_at_k_for_all:11                                                                        │
│                                                                                                  │
│    8 │   returns dict with recall@k and mrr                                                      │
│    9 │   """                                                                                     │
│   10 │   N, D = target_matrix.shape                                                              │
│ ❱ 11 │   assert query_matrix.shape[0] == N                                                       │
│   12 │   # compute similarity via dot product (rows normalized)                                  │
│   13 │   # sims matrix could be large O(N^2) but N=312 here is fine. For larger N, do batch.     │
│   14 │   sims = query_matrix @ target_matrix.T  # (N, N)                                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AssertionError

In [23]:
# --- Fixed evaluation cell: supports query_matrix with different size than target_matrix ---
import numpy as np
from collections import defaultdict

# fixed recall/MRR function: query_matrix can be MxD, target_matrix NxD
def recall_and_mrr(query_matrix: np.ndarray, target_matrix: np.ndarray, ids_list, topk_list=[1,5,10], gt_sets_local=None):
    """
    query_matrix: (M, D)
    target_matrix: (N, D)
    ids_list: list of length N containing ids matched to rows of target_matrix
    gt_sets_local: dict mapping id -> set(of relevant ids). If None, function will return None.
    Returns dict of metrics and number of valid queries.
    """
    M, D = query_matrix.shape
    N, D2 = target_matrix.shape
    assert D == D2, "dimension mismatch"
    if gt_sets_local is None:
        raise ValueError("gt_sets_local (ground truth dict) is required")

    sims_block = query_matrix @ target_matrix.T  # shape (M, N)
    # For each query we should avoid returning itself as top result if the id exists in ids_list.
    # But query_matrix may be a subset; we need mapping from query index -> query id.
    # We'll assume queries are in the same order as some provided list 'query_ids' when calling from outside.
    # To be flexible, we'll accept that caller will provide 'query_ids_for_matrix' if needed.
    results = {k: 0 for k in topk_list}
    mrr_sum = 0.0
    valid_queries = 0

    # We'll expect the caller to pass query_ids_for_matrix (list length M) to identify self-hits
    # To keep usage simple here, we'll build an index map from id->row index in target_matrix
    id_to_idx = {str(pid): idx for idx, pid in enumerate(ids_list)}

    for qi in range(M):
        qid = query_ids_for_matrix[qi]  # must be defined in caller scope
        gt = gt_sets_local.get(qid, set())
        if not gt:
            continue
        valid_queries += 1
        row = sims_block[qi].copy()
        # if qid exists in target_matrix ids, mask its self-similarity
        if qid in id_to_idx:
            row[id_to_idx[qid]] = -999.0
        idxs = np.argsort(-row)  # indices into target_matrix
        hit_rank = None
        for rank, idx in enumerate(idxs, start=1):
            cand_id = ids_list[int(idx)]
            if cand_id in gt:
                hit_rank = rank
                break
        if hit_rank is not None:
            for k in topk_list:
                if hit_rank <= k:
                    results[k] += 1
            mrr_sum += 1.0 / float(hit_rank)
    if valid_queries == 0:
        out = {f"recall@{k}": None for k in topk_list}
        out["MRR"] = None
        out["valid_queries"] = 0
        return out
    out = {}
    for k in topk_list:
        out[f"recall@{k}"] = results[k] / valid_queries
    out["MRR"] = mrr_sum / valid_queries
    out["valid_queries"] = valid_queries
    return out

# --- Prepare matrices and ids ---
# combined_matrix, text_matrix, img_matrix, ids (target ids) and gt_sets must already exist in memory
# If not, load them:
try:
    combined_matrix
except NameError:
    combined_matrix = np.load(str(INDEX_PATH), allow_pickle=True)["matrix"].astype(np.float32)
try:
    ids
except NameError:
    ids = np.load(str(INDEX_PATH), allow_pickle=True)["ids"].tolist()
try:
    text_matrix
except NameError:
    raise RuntimeError("text_matrix (text_emb_new) missing — rerun the chunked text embedding cell first.")
try:
    img_matrix
except NameError:
    img_matrix = np.zeros_like(text_matrix)

# normalize all
def normalize_rows(a):
    norms = np.linalg.norm(a, axis=1, keepdims=True)
    norms[norms==0] = 1.0
    return a / norms

combined_matrix = normalize_rows(combined_matrix)
text_matrix = normalize_rows(text_matrix)
img_matrix = normalize_rows(img_matrix)

# --- Evaluate on full-catalog queries (M = N) ---
# for full-catalog evaluation, query_ids_for_matrix is the same as ids
query_ids_for_matrix = ids  # required by recall_and_mrr
print("Evaluating full-catalog (each product queries the catalog)...")
combined_res = recall_and_mrr(combined_matrix, combined_matrix, ids, topk_list=[1,5,10], gt_sets_local=gt_sets)
print("Combined:", combined_res)
text_res = recall_and_mrr(text_matrix, combined_matrix, ids, topk_list=[1,5,10], gt_sets_local=gt_sets)
print("Text-only:", text_res)
img_res = recall_and_mrr(img_matrix, combined_matrix, ids, topk_list=[1,5,10], gt_sets_local=gt_sets)
print("Image-only:", img_res)

# --- Per-category breakdown using subset queries ---
# Build cat->pid mapping (category_tokens from earlier cell)
cat_to_pids = defaultdict(list)
for pid, toks in category_tokens.items():
    if not toks:
        continue
    primary = sorted(toks)[0]
    cat_to_pids[primary].append(pid)

cat_scores = []
for cat, pids in cat_to_pids.items():
    if len(pids) < 5:
        continue
    # build query matrix (M x D) from indices of these pids using combined_matrix rows
    q_idxs = [id_to_index[pid] for pid in pids if pid in id_to_index]
    if not q_idxs:
        continue
    qmat = combined_matrix[q_idxs]
    # define query ids list for this qmat (must set global name query_ids_for_matrix used by function)
    query_ids_for_matrix = [ids[i] for i in q_idxs]
    res = recall_and_mrr(qmat, combined_matrix, ids, topk_list=[5], gt_sets_local=gt_sets)
    if res and res.get("valid_queries",0) > 0:
        cat_scores.append((cat, len(pids), res["recall@5"]))
# sort and print
cat_scores_sorted = sorted(cat_scores, key=lambda x: -x[2])
print("Top categories by recall@5 (combined):")
for cat, cnt, score in cat_scores_sorted[:15]:
    print(f"{cat}: items={cnt}, recall@5={score:.3f}")


Evaluating full-catalog (each product queries the catalog)...
Combined: {'recall@1': 0.8621794871794872, 'recall@5': 0.9807692307692307, 'recall@10': 0.9871794871794872, 'MRR': 0.9166361326608706, 'valid_queries': 312}
Text-only: {'recall@1': 0.8621794871794872, 'recall@5': 0.9743589743589743, 'recall@10': 0.9903846153846154, 'MRR': 0.9139217610331918, 'valid_queries': 312}
Image-only: {'recall@1': 0.8557692307692307, 'recall@5': 0.9583333333333334, 'recall@10': 0.9743589743589743, 'MRR': 0.9053998558998558, 'valid_queries': 312}
Top categories by recall@5 (combined):
clothing & closet storage: items=11, recall@5=1.000
chairs: items=15, recall@5=1.000
doormats: items=20, recall@5=1.000
furniture: items=68, recall@5=1.000
bathroom furniture: items=5, recall@5=1.000
bedroom furniture: items=13, recall@5=1.000
bathroom hardware: items=17, recall@5=1.000
accessories: items=5, recall@5=1.000
end tables: items=21, recall@5=1.000
baby products: items=6, recall@5=1.000
dining room furniture: i

In [15]:
# Build text list and image path list from `products`.
# products must exist from earlier cells and have fields 'title','description','id','local_image','meta'
texts = [(p.get("title","") + ". " + (p.get("description") or "")) for p in products]
image_paths = [p.get("local_image") for p in products]

print("Number of products:", len(products))
print("Number of text items:", len(texts))
print("Number of image paths:", len(image_paths))
print("Example image path (first 5):", image_paths[:5])

# Compute text embeddings
text_emb = compute_text_embeddings(texts, batch_size=64)
print("Text embeddings shape:", text_emb.shape)
text_emb = l2_normalize_rows(text_emb)

# Compute image embeddings
img_emb = compute_image_embeddings(image_paths, batch_size=32)
print("Image embeddings shape:", img_emb.shape)
img_emb = l2_normalize_rows(img_emb)


Number of products: 312
Number of text items: 312
Number of image paths: 312
Example image path (first 5): ['/kaggle/working/images/02593e81-5c09-5069-8516-b0b29f439ded.jpg', '/kaggle/working/images/5938d217-b8c5-5d3e-b1cf-e28e340f292e.jpg', '/kaggle/working/images/b2ede786-3f51-5a45-9a5b-bcf856958cd8.jpg', '/kaggle/working/images/8fd9377b-cfa6-5f10-835c-6b8eca2816b5.jpg', '/kaggle/working/images/bdc9aa30-9439-50dc-8e89-213ea211d66a.jpg']


text batches:   0%|          | 0/5 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (379 > 77). Running this sequence through the model will result in indexing errors


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:12                                                                             │
│                                                                                                  │
│    9 print("Example image path (first 5):", image_paths[:5])                                     │
│   10                                                                                             │
│   11 # Compute text embeddings                                                                   │
│ ❱ 12 text_emb = compute_text_embeddings(texts, batch_size=64)                                    │
│   13 print("Text embeddings shape:", text_emb.shape)                                             │
│   14 text_emb = l2_normalize_rows(text_emb)                                                      │
│   15                                                                                             │
│                                                                                                  │
│ in compute_text_embeddings:32                                                                    │
│                                                                                                  │
│   29 │   │   for i in tqdm(range(0, len(texts), batch_size), desc="text batches"):               │
│   30 │   │   │   batch = texts[i:i+batch_size]                                                   │
│   31 │   │   │   inputs = processor(text=batch, return_tensors="pt", padding=True).to(device)    │
│ ❱ 32 │   │   │   emb = model.get_text_features(**inputs)   # (B, D)                              │
│   33 │   │   │   all_emb.append(emb.cpu().numpy())                                               │
│   34 │   if not all_emb:                                                                         │
│   35 │   │   return np.zeros((0, model.config.projection_dim), dtype=np.float32)                 │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/transformers/models/clip/modeling_clip.py:1004 in        │
│ get_text_features                                                                                │
│                                                                                                  │
│   1001 │   │   )                                                                                 │
│   1002 │   │   return_dict = return_dict if return_dict is not None else self.config.use_return  │
│   1003 │   │                                                                                     │
│ ❱ 1004 │   │   text_outputs = self.text_model(                                                   │
│   1005 │   │   │   input_ids=input_ids,                                                          │
│   1006 │   │   │   attention_mask=attention_mask,                                                │
│   1007 │   │   │   position_ids=position_ids,                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1739 in _wrapped_call_impl    │
│                                                                                                  │
│   1736 │   │   if self._compiled_call_impl is not None:                                          │
│   1737 │   │   │   return self._compiled_call_impl(*args, **kwargs)  # type: ignore[misc]        │
│   1738 │   │   else:                                                                             │
│ ❱ 1739 │   │   │   return self._call_impl(*args, **kwargs)                                       │
│   1740 │                                                                                         │
│   1741 │   # torchrec tests the code consistency with the f

In [24]:
import json
from pathlib import Path
WORK_DIR = Path("/kaggle/working")
OUT_DIR = WORK_DIR / "index"
OUT_DIR.mkdir(parents=True, exist_ok=True)
index_path = OUT_DIR / "index.npz"
meta_path = OUT_DIR / "index.npz.meta.json"

# combination strategy: weighted avg. change these weights if desired.
w_text = 0.6
w_img = 0.4

combined = []
ids = []
metadata = {}
missing_image_count = 0

for i, p in enumerate(products):
    te = text_emb[i] if text_emb.shape[0] > i else np.zeros((model.config.projection_dim,), dtype=np.float32)
    ie = img_emb[i] if img_emb.shape[0] > i else np.zeros((model.config.projection_dim,), dtype=np.float32)
    if p.get("local_image") is None or (ie is None or ie.size == 0):
        v = te
        missing_image_count += 1
    else:
        v = (w_text * te + w_img * ie)
    # normalize
    v = v / (np.linalg.norm(v) + 1e-12)
    combined.append(v.astype(np.float32))
    pid = str(p.get("id") or p.get("uniq_id") or i)
    ids.append(pid)
    # metadata: keep provided meta dict (drop local_image to keep payload small)
    meta = p.get("meta", {}).copy()
    meta.pop("local_image", None)
    metadata[pid] = meta

matrix = np.vstack(combined)
print("Combined matrix shape:", matrix.shape)
print("Products without image (used text only):", missing_image_count)

# Save .npz and meta.json
np.savez_compressed(str(index_path), ids=np.array(ids, dtype=object), matrix=matrix)
with open(meta_path, "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

print("Saved index to:", index_path)
print("Saved metadata to:", meta_path)


Combined matrix shape: (312, 512)
Products without image (used text only): 0
Saved index to: /kaggle/working/index/index.npz
Saved metadata to: /kaggle/working/index/index.npz.meta.json


In [27]:
# quick verify by loading back and doing a sanity nearest neighbor for a sample prompt
import numpy as np
data = np.load(str(index_path), allow_pickle=True)
ids_loaded = data["ids"].tolist()
mat_loaded = data["matrix"]
print("Loaded ids:", len(ids_loaded))
print("Loaded matrix shape:", mat_loaded.shape)

# Example: compute embedding for sample prompt & find top-5 nearest (cosine)
sample_prompt = "cat"
# compute text embedding using CLIP processor + model
inputs = processor(text=[sample_prompt], return_tensors="pt", padding=True).to(device)
with torch.no_grad():
    q = model.get_text_features(**inputs).cpu().numpy()[0]
# normalize
q = q / (np.linalg.norm(q) + 1e-12)

# cosine via dot (matrix rows are normalized)
sims = (mat_loaded @ q).reshape(-1)
topk = 5
idxs = np.argsort(-sims)[:topk]
print("Top results (id,score):")
for ii in idxs:
    pid = ids_loaded[int(ii)]
    score = float(sims[int(ii)])
    print(pid, score)
    # print title from metadata for convenience
    try:
        print("  title:", metadata[pid].get("title")[:150])
    except Exception:
        pass


Loaded ids: 312
Loaded matrix shape: (312, 512)
Top results (id,score):
2631f7c4-fc61-5b16-9824-7acf7fd59994 0.6638836860656738
  title: Simple Deluxe Gaming Chair, Big and Tall Gamer Chair, Racing Style Adjustable Swivel Office Chair, Ergonomic Video Game Chairs with Headrest and Lumba
780dda1f-e7ef-598e-9c0e-6536c4b4c261 0.660207986831665
  title: Stylish Camping Ming's Mark RC4 Reversible Classical Patio Mat - 8' x 20', Green/Beige
37323128-75a9-578b-9378-79653bfd5b52 0.649467945098877
  title: FANYE Oversized 6 Seaters Modular Storage Sectional Sofa Couch for Home Apartment Office Living Room,Free Combination L/U Shaped Corduroy Upholstered 
20da3703-26f1-53e5-aa0b-a8104527d1bb 0.6392088532447815
  title: LOVMOR 30'' Bathroom Vanity Sink Base Cabine, Storage Cabinet with 3-Drawers on The Left, Suitable for Bathrooms, Kitchens, Laundry Rooms and Other Pl
20da3703-26f1-53e5-aa0b-a8104527d1bb 0.6392088532447815
  title: LOVMOR 30'' Bathroom Vanity Sink Base Cabine, Storage Cabinet wit

meow

In [26]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)
model_name = "openai/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_name).to(device)
processor = CLIPProcessor.from_pretrained(model_name)
model.eval()

def compute_text_embeddings(texts, batch_size=64):
    all_emb = []
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            inputs = processor(text=batch, return_tensors="pt", padding=True).to(device)
            emb = model.get_text_features(**inputs)
            all_emb.append(emb.cpu().numpy())
    if all_emb:
        return np.vstack(all_emb)
    return np.zeros((0, model.config.projection_dim), dtype=np.float32)

def compute_image_embeddings(paths, batch_size=32):
    all_emb = []
    with torch.no_grad():
        for i in range(0, len(paths), batch_size):
            batch = paths[i:i+batch_size]
            images = []
            for p in batch:
                if p is None:
                    images.append(Image.new("RGB",(224,224),color=(128,128,128)))
                else:
                    try:
                        img = Image.open(p).convert("RGB")
                    except Exception:
                        img = Image.new("RGB",(224,224),color=(128,128,128))
                    images.append(img)
            inputs = processor(images=images, return_tensors="pt", padding=True).to(device)
            emb = model.get_image_features(**inputs)
            all_emb.append(emb.cpu().numpy())
    if all_emb:
        return np.vstack(all_emb)
    return np.zeros((0, model.config.projection_dim), dtype=np.float32)

# Prepare lists
texts = [ (p["title"] + ". " + (p["description"] or "")) for p in products ]
image_paths = [ p.get("local_image") for p in products ]

print("Computing text embeddings...")
text_emb = compute_text_embeddings(texts, batch_size=64)
print("Computing image embeddings...")
img_emb = compute_image_embeddings(image_paths, batch_size=32)

# L2 normalize
def l2norm(a):
    if a.size == 0:
        return a
    norms = np.linalg.norm(a, axis=1, keepdims=True)
    norms[norms==0] = 1.0
    return a / norms

text_emb = l2norm(text_emb)
img_emb = l2norm(img_emb)

print("Shapes -> text:", text_emb.shape, "img:", img_emb.shape)


Using device: cpu


Token indices sequence length is longer than the specified maximum sequence length for this model (379 > 77). Running this sequence through the model will result in indexing errors


Computing text embeddings...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:47                                                                             │
│                                                                                                  │
│   44 image_paths = [ p.get("local_image") for p in products ]                                    │
│   45                                                                                             │
│   46 print("Computing text embeddings...")                                                       │
│ ❱ 47 text_emb = compute_text_embeddings(texts, batch_size=64)                                    │
│   48 print("Computing image embeddings...")                                                      │
│   49 img_emb = compute_image_embeddings(image_paths, batch_size=32)                              │
│   50                                                                                             │
│                                                                                                  │
│ in compute_text_embeddings:14                                                                    │
│                                                                                                  │
│   11 │   │   for i in range(0, len(texts), batch_size):                                          │
│   12 │   │   │   batch = texts[i:i+batch_size]                                                   │
│   13 │   │   │   inputs = processor(text=batch, return_tensors="pt", padding=True).to(device)    │
│ ❱ 14 │   │   │   emb = model.get_text_features(**inputs)                                         │
│   15 │   │   │   all_emb.append(emb.cpu().numpy())                                               │
│   16 │   if all_emb:                                                                             │
│   17 │   │   return np.vstack(all_emb)                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/transformers/models/clip/modeling_clip.py:1004 in        │
│ get_text_features                                                                                │
│                                                                                                  │
│   1001 │   │   )                                                                                 │
│   1002 │   │   return_dict = return_dict if return_dict is not None else self.config.use_return  │
│   1003 │   │                                                                                     │
│ ❱ 1004 │   │   text_outputs = self.text_model(                                                   │
│   1005 │   │   │   input_ids=input_ids,                                                          │
│   1006 │   │   │   attention_mask=attention_mask,                                                │
│   1007 │   │   │   position_ids=position_ids,                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1739 in _wrapped_call_impl    │
│                                                                                                  │
│   1736 │   │   if self._compiled_call_impl is not None:                                          │
│   1737 │   │   │   return self._compiled_call_impl(*args, **kwargs)  # type: ignore[misc]        │
│   1738 │   │   else:                                                                             │
│ ❱ 1739 │   │   │   return self._call_impl(*args, **kwargs)                                       │
│   1740 │                                                                                         │
│   1741 │   # torchrec tests the code consistency with the f

In [ ]:
# Combine vectors: weighted average (text-heavy by default)
w_text = 0.6
w_img = 0.4

combined = []
ids = []
metadata = {}

for i, p in enumerate(products):
    te = text_emb[i] if text_emb.shape[0] > i else np.zeros((model.config.projection_dim,), dtype=np.float32)
    ie = img_emb[i] if img_emb.shape[0] > i else np.zeros((model.config.projection_dim,), dtype=np.float32)
    if p.get("local_image") is None:
        v = te
    else:
        v = (w_text * te + w_img * ie)
    v = v / (np.linalg.norm(v) + 1e-12)
    combined.append(v.astype(np.float32))
    ids.append(str(p["id"]))
    # metadata should include all given columns (drop local path)
    meta = p["meta"].copy()
    meta.pop("local_image", None)
    meta.pop("images_field", None)
    metadata[str(p["id"])] = meta

matrix = np.vstack(combined)
print("Combined matrix shape:", matrix.shape)

OUT_DIR = WORK_DIR / "index"
OUT_DIR.mkdir(parents=True, exist_ok=True)
index_path = OUT_DIR / "index.npz"
meta_path = OUT_DIR / "index.npz.meta.json"

np.savez_compressed(str(index_path), ids=np.array(ids, dtype=object), matrix=matrix)
with open(meta_path, "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

print("Saved index:", index_path)
print("Saved metadata:", meta_path)
